In [1]:
import json, random, math, numpy as np
from datetime import datetime
from tqdm import tqdm
from string import Template
from langchain_openai import OpenAIEmbeddings
from scipy.spatial.distance import cosine
from shroom_classifier_v7 import ShroomClassifier

In [2]:
MODEL = "gpt-4"

TEMP = 1.2

DATE = datetime.utcnow().date().isoformat()

SAMPLE_SIZE = 64

EMBEDDINGS_MODEL = OpenAIEmbeddings()

CLASSIFIER = ShroomClassifier(model_name=MODEL, temperature=TEMP)

PSEUDO_DEMO_TEMPLATE = Template("""##
Input text: $src
Target text: $tgt
Generated text: $hyp
Answer: $label
""")

In [3]:
def binary_negative_entropy(p, epsilon=1e-10):
    # Adjust probabilities to avoid log(0)
    p = np.clip(p, epsilon, 1 - epsilon)
    return p * np.log(p) + (1 - p) * np.log(1 - p)

In [4]:
def cos_sim(v1, v2):
    return 1 - cosine(v1, v2)

In [5]:
def phi(dp, classification):
    serialization = f'{dp["hyp"]} {dp["tgt"]} {dp["src"]} {dp["tgt"]} {classification["label"]}'
    return EMBEDDINGS_MODEL.embed_query(serialization)

In [6]:
def selection_metric(p, S, l=0.2):
    return p['F_CLS'] - (l * max([ cos_sim(p["phi"], s["phi"]) for s in S ]))

In [7]:
def generate_pseudo_demos(datapoints):
    pseudo_demos = []
    for dp in tqdm(datapoints):  
        classification = CLASSIFIER.classify(dp)
        pseudo_demos.append(
            {
                "datapoint": dp,
                "classification": classification,
                "F_CLS": binary_negative_entropy(classification["p(Hallucination)"]),
                "phi": phi(dp, classification) 
            }
        )
    return pseudo_demos

In [8]:
def select_pseudo_demos(pseudo_demos, K=3):
    pool = pseudo_demos
    selections = []
    for k in range(K):
        if k == 0:
            sk = max(pool, key=lambda x: x['F_CLS'])
        else:
            sk = max(pool, key=lambda x: selection_metric(x, selections))
        selections.append(sk)
        pool.remove(sk)
    return selections

In [9]:
def serialize_selected_pseudo_demos(pseudo_demos):
    return ''.join([ PSEUDO_DEMO_TEMPLATE.substitute(
        {
            "hyp": pd["datapoint"]["hyp"],
            "src": pd["datapoint"]["src"],        
            "tgt": pd["datapoint"]["tgt"],        
            "label": pd["classification"]["label"],    
        }
    ) for pd in pseudo_demos])

In [10]:
dataset = json.load(open('reference/train.model-agnostic.json', 'r'))
dm_pds = generate_pseudo_demos(random.sample([ dp for dp in dataset if dp['task'] == "DM" ], SAMPLE_SIZE))
pg_pds = generate_pseudo_demos(random.sample([ dp for dp in dataset if dp['task'] == "PG" ], SAMPLE_SIZE))
mt_pds = generate_pseudo_demos(random.sample([ dp for dp in dataset if dp['task'] == "MT" ], SAMPLE_SIZE))

  0%|          | 0/64 [00:00<?, ?it/s]

100%|██████████| 64/64 [01:18<00:00,  1.22s/it]


In [11]:
dm_pds_pos = [ pd for pd in dm_pds if pd["classification"]["label"] == "Hallucination" ]
dm_pds_neg = [ pd for pd in dm_pds if pd["classification"]["label"] == "Not Hallucination" ]
selected_dm_pds = select_pseudo_demos(dm_pds_pos) + select_pseudo_demos(dm_pds_neg)
dm_demos = serialize_selected_pseudo_demos(selected_dm_pds)
print(dm_demos)

##
Input text: Instead , the shares are held by a <define> superentity </define> called Cede & Company in an arrangement administered by the Depositary Trust and Clearing Corporation .
Target text: An entity comprising several lesser entities.
Generated text: (finance) A company that owns shares in another company.
Answer: Hallucination
##
Input text: Now though Cider uſed in my Method ſhould not ferment at all , till it come into the Bottle , and then but a little ; yet the Cauſe of Fermentation being in a great Degree taken away , the reſt can do no conſiderable Harm to thoſe who drink it , [ ... ] It is in your Power to give the Cider juſt as much <define> fret </define> as you pleaſe , and no more ; and that by ſeveral ways : For either you may bottle it ſooner or later , as you pleaſe : Or you may bottle it from two Taps in your Veſſel , and that from the higher Tap will have leſs Fret , and the lower more : [ ... ]
Target text: Agitation of the surface of a fluid by fermentation 

In [12]:
pg_pds_pos = [ pd for pd in pg_pds if pd["classification"]["label"] == "Hallucination" ]
pg_pds_neg = [ pd for pd in pg_pds if pd["classification"]["label"] == "Not Hallucination" ]
selected_pg_pds = select_pseudo_demos(pg_pds_pos) + select_pseudo_demos(pg_pds_neg)
pg_demos = serialize_selected_pseudo_demos(selected_pg_pds)
print(pg_demos)

##
Input text: What'd they do?
Target text: 
Generated text: What'd you think they'd be doing?
Answer: Hallucination
##
Input text: Dead or alive, you're coming with me.
Target text: 
Generated text: Dead or alive, you're comin 'into with me.
Answer: Hallucination
##
Input text: Maybe you can use one of these.
Target text: 
Generated text: Maybe you can use one of these cuffs.
Answer: Hallucination
##
Input text: Hey, Frankie.
Target text: 
Generated text: Hey, frankie.
Answer: Not Hallucination
##
Input text: I seem to come back to the present when I finish some chapter in the lives of people that I'm tracking.
Target text: 
Generated text: I seem to come back to the present when I'm finishing some chapter in the lives of people that I'm tracking.
Answer: Not Hallucination
##
Input text: Could be dangerous.
Target text: 
Generated text: It could be dangerously.
Answer: Not Hallucination



In [13]:
mt_pds_pos = [ pd for pd in mt_pds if pd["classification"]["label"] == "Hallucination" ]
mt_pds_neg = [ pd for pd in mt_pds if pd["classification"]["label"] == "Not Hallucination" ]
selected_mt_pds = select_pseudo_demos(mt_pds_pos) + select_pseudo_demos(mt_pds_neg)
mt_demos = serialize_selected_pseudo_demos(selected_mt_pds)
print(mt_demos)

##
Input text: Том сказал, когда вернётся?
Target text: Did Tom say when he'd be back?
Generated text: Tom said when he got back?
Answer: Hallucination
##
Input text: Он нам сейчас не нужен.
Target text: We don't need it now.
Generated text: We don't need him now.
Answer: Hallucination
##
Input text: Едва завидев меня, Том рассмеялся.
Target text: As soon as Tom saw me, he started laughing.
Generated text: Two jealous of me, Tom laughed.
Answer: Hallucination
##
Input text: Я получаю всё, что хочу.
Target text: I get everything I want.
Generated text: I get everything I want.
Answer: Not Hallucination
##
Input text: День подошёл к концу. Благодаря Вашим своевременным усилиям, люди, которые стремились помочь Вам в Ваших трудах, не успели подвергнуть себя опасности. В Театре, Маски репетируют новую пантомиму.
Target text: The day has reached its end. Thanks to your timely efforts, people who intended to help you in your labours managed not to throw themselves into danger. In the theater,